Reference: https://music-encoding.org/guidelines/v4/data-types/data.accidental.written.html

In [4]:
import numpy as np
from typing import List, Tuple
from IPython.display import display, HTML
import verovio
import pandas as pd


df = pd.read_csv('speech1.tsv', sep='\t')

A4_FREQUENCY = 440

def convert_frequencies(df: pd.DataFrame) -> Tuple[List[str], List[int], List[str]]:
    frequencies = df['Frequency (Hz)'].tolist()
    def acoustical_pitch_name(midi_num: int) -> str:
        pitch_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        pitch_idx = midi_num % 12
        pitch_name = pitch_names[pitch_idx]
        octave = (midi_num - 12) // 12
        return f"{pitch_name}{octave}"

    def midi_value(log_freq_ratio: float) -> int:
        return int(round(12 * log_freq_ratio + 69))

    def midi_cent_value(log_freq_ratio: float) -> int:
        return int(round(69 * 100 + 1200 * log_freq_ratio))

    log_freq_ratios = [np.log2(freq / A4_FREQUENCY) for freq in frequencies]
    midi_values = [midi_value(log_freq_ratio) for log_freq_ratio in log_freq_ratios]
    midi_cent_values = [midi_cent_value(log_freq_ratio) for log_freq_ratio in log_freq_ratios]
    pitch_names = [acoustical_pitch_name(midi_value) for midi_value in midi_values]
    midi_cent_deviations = [((midi_cent_value - midi_value * 100) + 50) % 100 - 50 for midi_value, midi_cent_value in zip(midi_values, midi_cent_values)]
    midi_cent_deviations = [f'+{x}' if x >= 0 else f'{x}' for x in midi_cent_deviations]

    return pitch_names, midi_values, midi_cent_deviations



pitch_names, midi_values, midi_cent_deviations = convert_frequencies(df)

# print(midi_values)
# print(midi_cent_deviations)
# print(pitch_names)

def pitch_name_to_step_octave_alter(pitch_name: str) -> Tuple[str, int, int]:
    step = pitch_name[0]
    alter = 1 if '#' in pitch_name else -1 if 'b' in pitch_name else 0
    octave = int(pitch_name[-1])
    return step, octave, alter

def pitch_name_and_cent_deviation_to_step_octave_alter(pitch_name: str, cent_deviation: str) -> Tuple[str, int, str]:
    step = pitch_name[0]
    octave = int(pitch_name[-1])
    deviation = int(cent_deviation)
    alter = 'n'
    if '#' in pitch_name:
        alter = 's'
        if deviation >= 25:
            alter = '3qs'
        elif deviation > 0:
            alter = '1qs'
    elif 'b' in pitch_name:
        alter = 'f'
        if deviation <= -25:
            alter = '3qf'
        elif deviation < 0:
            alter = '1qf'
    else:
        if deviation >= 25:
            alter = '1qs'
        elif deviation > 0:
            alter = '1qs'
        elif deviation <= -25:
            alter = '1qf'
        elif deviation < 0:
            alter = '1qf'
    return step, octave, alter

notes_and_alters = [pitch_name_and_cent_deviation_to_step_octave_alter(pitch_name, cent_deviation) for pitch_name, cent_deviation in zip(pitch_names, midi_cent_deviations)]

#print(notes_and_alters)

def create_mei_string(notes_and_alters: List[Tuple[str, int, str]], midi_values: List[int], staff: str) -> str:
    staff_defs = ''
    if 'G' in staff:
        staff_defs += '<staffDef n="1" lines="5" clef.shape="G" clef.line="2"/>\n'
    if 'F' in staff:
        staff_defs += '<staffDef n="2" lines="5" clef.shape="F" clef.line="4"/>\n'
    
    mei = f'''
    <?xml version="1.0" encoding="UTF-8"?>
    <mei xmlns="http://www.music-encoding.org/ns/mei" meiversion="4.0.1">
        <music>
            <body>
                <mdiv>
                    <score>
                        <scoreDef keysig="0" mode="major">
                            <staffGrp n="1">
                                {staff_defs}
                            </staffGrp>
                        </scoreDef>
                        <section>
    '''
    for i, ((step, octave, alter), midi_value) in enumerate(zip(notes_and_alters, midi_values)):
        mei += f'<measure n="{i+1}" right="invis">\n'
        if 'G' in staff:
            mei += '<staff n="1">\n<layer n="1">\n'
            if octave >= 4 or (octave == 3 and step == 'B'):
                mei += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            mei += '</layer>\n</staff>\n'
        if 'F' in staff:
            mei += '<staff n="2">\n<layer n="1">\n'
            if octave < 4 or (octave == 4 and step == 'C' and alter in ['f', '1qf', '3qf']):
                mei += f'''
                                        <note dur="4" oct="{octave}" pname="{step.lower()}" pnum="{midi_value}" stem.dir="down" accid="{alter}" stem.visible="false"/>
                '''
            mei += '</layer>\n</staff>\n'
        mei += '</measure>\n'
    mei += '''
                        </section>
                    </score>
                </mdiv>
            </body>
        </music>
    </mei>
    '''
    return mei


mei = create_mei_string(notes_and_alters, midi_values.copy(), staff='GF')

frequencies = df['Frequency (Hz)'].tolist()

# create a DataFrame
df_para = pd.DataFrame({
    'Frequency (Hz)': frequencies,
    'MIDI Value': midi_values,
    'MIDI Cent Deviation': midi_cent_deviations,
    'Pitch Name': pitch_names,
    'Note': [x[0] for x in notes_and_alters],
    'Octave': [x[1] for x in notes_and_alters],
    'Alter': [x[2] for x in notes_and_alters],
})

display(df_para)

# Create a VerovioToolkit object
vrvToolkit = verovio.toolkit()

# Set the rendering options
options = {
    "inputFormat": "mei",
    "scale": 50,
    "adjustPageHeight": True,
    "noHeader": True,
    "border": 0,
    "pageHeight": 5400,
    "pageWidth": 1800,
    "staffLineWidth": 0.15,
    "systemDividerLineWidth": 1.5
}

# Render the music notation and get the HTML code
htmlCode = vrvToolkit.renderData(mei, options=options)

# Display the HTML code in the notebook
display(HTML(htmlCode))


,Frequency (Hz),MIDI Value,MIDI Cent Deviation,Pitch Name,Note,Octave,Alter
0,97.368421,43,-11,G2,G,2,1qf
1,105.263158,44,+24,G#2,G,2,1qs
2,110.526316,45,+8,A2,A,2,1qs
3,194.736842,55,-11,G3,G,3,1qf
4,202.631579,56,-42,G#3,G,3,s
5,210.526316,56,+24,G#3,G,3,1qs
6,215.789474,57,-33,A3,A,3,1qf
7,223.684211,57,+29,A3,A,3,1qs
8,228.947368,58,-31,A#3,A,3,s
9,292.105263,62,-9,D4,D,4,1qf


[Error] Unsupported option 'border'
[Error] Unsupported option 'inputFormat'
[Error] Unsupported option 'noHeader'
[Error] Unsupported option 'systemDividerLineWidth'
[Warning] No header found in the MEI data, trying to proceed...
